In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install geopandas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import shapely

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/dataset/madison/'

## Load dataset

In [ ]:
objects = pd.read_csv(path + '/input/objects_embeddings.csv')
objects[['embedding_ada']] = objects[['embedding_ada']].applymap(lambda s: np.array(eval(s)))
objects[['embedding_large']] = objects[['embedding_large']].applymap(lambda s: np.array(eval(s)))
objects.head()

,objectid,geometry,source,geom_type,n_tokens,embedding_ada,embedding_large
0,"road_(6231530142, 3861040918)","LINESTRING (-89.4719 43.05, -89.472 43.05)",link,LineString,21,"[-0.014463399536907673, -0.004633771721273661,...","[0.01921754889190197, 0.01929466612637043, -0...."
1,"road_(53591318, 1179412798)","LINESTRING (-89.477 43.0693, -89.477 43.0681)",link,LineString,22,"[-0.014376881532371044, -0.004993679467588663,...","[0.021175693720579147, 0.015708766877651215, -..."
2,"road_(1178722373, 1178722211)","LINESTRING (-89.4626 43.0553, -89.4627 43.0554)",link,LineString,24,"[-0.015243860892951488, -0.004274553153663874,...","[0.011615517549216747, 0.02089725434780121, -0..."
3,"road_(53430628, 53468282)","LINESTRING (-89.3045 43.0246, -89.3053 43.0252)",link,LineString,24,"[-0.016633406281471252, -0.0014020365197211504...","[0.0076748463325202465, 0.030930323526263237, ..."
4,"road_(3444117858, 6680557104)","LINESTRING (-89.5081 43.0066, -89.5072 43.006)",link,LineString,23,"[-0.010978729464113712, -0.005506434012204409,...","[0.03258766606450081, 0.024168409407138824, -0..."


In [ ]:

relations = pd.read_csv(path+'input/relations.csv', usecols=['subjectid', 'objectid', 'predicate', 'relate', 'idx'])
# relations.to_csv('/content/drive/MyDrive/dataset/madison/output/relations_update.csv', index=False)
# relations = pd.read_csv('/content/drive/MyDrive/dataset/madison/output/relations_update.csv')
relations.head()

,objectid,subjectid,predicate,relate,idx
0,"road_(53496379, 5657142535)","road_(5656899646, 53496379)",contains,101F00FF2,0
1,"road_(2634716954, 8997363243)","road_(8997363242, 2634716954)",contains,101F00FF2,1
2,poi_way/1134399612,poi_way/1134399613,contains,1F1FFFFF2,2
3,"road_(1998162788, 1998162839)",poi_way/257174438,contains,101FFFFF2,3
4,"road_(3420767254, 3590603510)",poi_way/287637061,contains,101FFFFF2,4


In [ ]:
relations = pd.merge(relations, objects[['objectid', 'geometry', 'source', 'geom_type','n_tokens']].rename(columns={'objectid': 'subjectid',
                                                        'geometry': 'geometry_subject',
                                                        'source': 'source_subject',
                                                        'geom_type': 'geom_type_subject',
                                                          'n_tokens': 'n_tokens_subject'
                                                        }), on='subjectid', how='inner')
relations = pd.merge(relations, objects[['objectid', 'geometry', 'source', 'geom_type', 'n_tokens']], on='objectid', how='inner')
# relations = gpd.GeoDataFrame(relations, geometry='geometry', crs=4326)
# relations['geometry_subject'] = gpd.GeoSeries(relations['geometry_subject'])

# relations['relate'] = relations['geometry_subject'].relate(relations['geometry'])
relations.head()

,objectid,subjectid,predicate,relate,idx,geometry_subject,source_subject,geom_type_subject,n_tokens_subject,geometry,source,geom_type,n_tokens
0,"road_(53496379, 5657142535)","road_(5656899646, 53496379)",contains,101F00FF2,0,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24
1,"road_(53496379, 5657142535)","road_(53496379, 5656899646)",contains,101F00FF2,31,"LINESTRING (-89.3386 43.0922, -89.3397 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24
2,"road_(5657142535, 53496379)","road_(5656899646, 53496379)",contains,101F00FF2,202,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24
3,"road_(5657142535, 53496379)","road_(53496379, 5656899646)",contains,101F00FF2,54,"LINESTRING (-89.3386 43.0922, -89.3397 43.0922)",link,LineString,24,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24
4,"road_(5657142535, 53496379)",census_550250020003,contains,102FF1FF2,180,"POLYGON ((-89.3515 43.0928, -89.346 43.0939, -...",cbg,Polygon,109,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24


In [ ]:
relate_df = relations[['geom_type_subject', 'predicate', 'geom_type']].drop_duplicates().reset_index(drop=True)
relate_df['rid'] = relate_df.index
relations = pd.merge(relations, relate_df, on=['geom_type_subject', 'predicate', 'geom_type'], how='left')

In [ ]:
relate_df

,geom_type_subject,predicate,geom_type,rid
0,LineString,contains,LineString,0
1,Polygon,contains,LineString,1
2,Point,disjoint,LineString,2
3,LineString,disjoint,Polygon,3
4,Point,disjoint,Polygon,4
5,LineString,within,Polygon,5
6,LineString,crosses,Polygon,6
7,Point,within,Polygon,7
8,Point,touches,Polygon,8
9,LineString,touches,LineString,9


In [ ]:
relations[['objectid', 'predicate']].groupby(['predicate']).count()

,objectid
predicate,
contains,1125
crosses,675
disjoint,2025
equals,675
overlaps,450
touches,1800
within,1125


In [ ]:

# intersection = pd.read_csv('/content/drive/MyDrive/dataset/madison/input/intersection.csv')
# intersection = intersection[(intersection['subjectid'].isin(set(objects['objectid'])))&
#                       (intersection['objectid'].isin(set(object_relation_embed['objectid'])))]
# intersection = pd.merge(intersection, objects[['objectid', 'geom_type']].rename(columns={'objectid': 'subjectid',
#                                                                                          'geom_type': 'geom_type_subject'}),
#                         on='subjectid', how='left')
# intersection = pd.merge(intersection, objects[['objectid', 'geom_type']],
#                         on='objectid', how='left')
# intersection.head()

## Train / Test split

In [ ]:
# relations = relations.sample(frac=1, random_state=42).reset_index(drop=True).sort_values(by=['rid']).reset_index(drop=True)
# relations['idx'] = relations.index
# relations_first = (relations[['geom_type_subject', 'predicate', 'geom_type', 'idx']]
#                    .drop_duplicates(subset=['geom_type_subject', 'predicate', 'geom_type'])
#                    .reset_index(drop=True))
# relations = pd.merge(relations, relations_first,
#                      on=['geom_type_subject', 'predicate', 'geom_type'],
#                      how='left')

# relations['idx'] = relations['idx_x'] - relations['idx_y']

# # n_idx = 5
# # relations['idx'] = np.mod(relations['idx'], n_idx)

# # relations = relations.drop(columns=['idx_x', 'idx_y']).sample(frac=1, random_state=42).reset_index(drop=True)
# print(relations.shape[0], relations[['subjectid', 'objectid', 'predicate']].drop_duplicates().shape[0])
# relations.head()

In [ ]:
# relations  = relations.drop(columns=['embedding_subject', 'embedding'])
relations = pd.merge(relations, objects[['objectid', 'embedding_ada', 'embedding_large']].rename(columns={'objectid': 'subjectid',

                                                        'embedding_ada': 'embedding_subject_ada',
                                                        'embedding_large': 'embedding_subject_large'
                                                        }), on='subjectid', how='left')
relations = pd.merge(relations, objects[['objectid', 'embedding_ada', 'embedding_large']],
                     on='objectid', how='left')

relations.head()

,objectid,subjectid,predicate,relate,idx,geometry_subject,source_subject,geom_type_subject,n_tokens_subject,geometry,source,geom_type,n_tokens,rid,embedding_subject_ada,embedding_subject_large,embedding_ada,embedding_large
0,"road_(53496379, 5657142535)","road_(5656899646, 53496379)",contains,101F00FF2,0,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24,0,"[-0.012084594927728176, -0.0009365046862512827...","[0.026674335822463036, 0.026426058262586594, -...","[-0.01336305309087038, -0.002070763148367405, ...","[0.021892955526709557, 0.0262684877961874, -0...."
1,"road_(53496379, 5657142535)","road_(53496379, 5656899646)",contains,101F00FF2,31,"LINESTRING (-89.3386 43.0922, -89.3397 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24,0,"[-0.012219272553920746, -0.002612584037706256,...","[0.02258148044347763, 0.024502981454133987, -0...","[-0.01336305309087038, -0.002070763148367405, ...","[0.021892955526709557, 0.0262684877961874, -0...."
2,"road_(5657142535, 53496379)","road_(5656899646, 53496379)",contains,101F00FF2,202,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24,0,"[-0.012084594927728176, -0.0009365046862512827...","[0.026674335822463036, 0.026426058262586594, -...","[-0.01222349051386118, -0.0006331487675197423,...","[0.020967820659279823, 0.02303224988281727, -0..."
3,"road_(5657142535, 53496379)","road_(53496379, 5656899646)",contains,101F00FF2,54,"LINESTRING (-89.3386 43.0922, -89.3397 43.0922)",link,LineString,24,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24,0,"[-0.012219272553920746, -0.002612584037706256,...","[0.02258148044347763, 0.024502981454133987, -0...","[-0.01222349051386118, -0.0006331487675197423,...","[0.020967820659279823, 0.02303224988281727, -0..."
4,"road_(5657142535, 53496379)",census_550250020003,contains,102FF1FF2,180,"POLYGON ((-89.3515 43.0928, -89.346 43.0939, -...",cbg,Polygon,109,"LINESTRING (-89.3387 43.0922, -89.3386 43.0922)",link,LineString,24,1,"[-0.0005961718852631748, -0.002250827616080641...","[0.02854921482503414, 0.013134698383510113, -0...","[-0.01222349051386118, -0.0006331487675197423,...","[0.020967820659279823, 0.02303224988281727, -0..."


In [ ]:
relations['output'] = '(' + relations['geom_type_subject'] + ', ' + relations['predicate']+ ', ' + relations['geom_type'] + ')'

In [ ]:
relations_example = relations[relations['idx']>=200].reset_index(drop=True)

relations_test = relations[relations['idx']<40].reset_index(drop=True)

relations_train = relations[(relations['idx']>=40)&(relations['idx']<200)].reset_index(drop=True)

In [ ]:
relations_test.sort_values(by=['rid', 'idx']).head()

,objectid,subjectid,predicate,relate,idx,geometry_subject,source_subject,geom_type_subject,n_tokens_subject,geometry,source,geom_type,n_tokens,rid,embedding_subject_ada,embedding_subject_large,embedding_ada,embedding_large,output
0,"road_(53496379, 5657142535)","road_(5656899646, 53496379)",contains,101F00FF2,0,"LINESTRING (-89.3397 43.0922, -89.3386 43.0922)",link,LineString,24,"LINESTRING (-89.3386 43.0922, -89.3387 43.0922)",link,LineString,24,0,"[-0.012084594927728176, -0.0009365046862512827...","[0.026674335822463036, 0.026426058262586594, -...","[-0.01336305309087038, -0.002070763148367405, ...","[0.021892955526709557, 0.0262684877961874, -0....","(LineString, contains, LineString)"
2,"road_(2634716954, 8997363243)","road_(8997363242, 2634716954)",contains,101F00FF2,1,"LINESTRING (-89.3554 43.1351, -89.3554 43.1357)",link,LineString,24,"LINESTRING (-89.3554 43.1357, -89.3554 43.1355)",link,LineString,24,0,"[-0.013053328730165958, -0.0032871149014681578...","[0.013792595826089382, 0.02975725382566452, -0...","[-0.01368382666260004, -0.001345838070847094, ...","[0.013515212573111057, 0.028356211259961128, -...","(LineString, contains, LineString)"
3,poi_way/1134399612,poi_way/1134399613,contains,1F1FFFFF2,2,"LINESTRING (-89.4524 43.0716, -89.4521 43.0716...",poi,LineString,57,"LINESTRING (-89.4524 43.0715, -89.4521 43.0715...",poi,LineString,46,0,"[-0.018740536645054817, -0.0004815085558220744...","[0.028172552585601807, 0.027236536145210266, -...","[-0.01892492175102234, -0.004294237121939659, ...","[0.02348930761218071, 0.02849321812391281, -0....","(LineString, contains, LineString)"
6,"road_(1998162788, 1998162839)",poi_way/257174438,contains,101FFFFF2,3,"LINESTRING (-89.3392 43.0637, -89.3399 43.0641...",poi,LineString,134,"LINESTRING (-89.338 43.0659, -89.3378 43.0659)",link,LineString,23,0,"[-0.016371171921491623, -0.0005665214848704636...","[0.04231448471546173, 0.02749074622988701, -0....","[-0.011485625058412552, -0.0073748365975916386...","[0.018955102190375328, 0.020605383440852165, -...","(LineString, contains, LineString)"
10,"road_(3420767254, 3590603510)",poi_way/287637061,contains,101FFFFF2,4,"LINESTRING (-89.5168 43.0606, -89.5168 43.0616...",poi,LineString,57,"LINESTRING (-89.5175 43.0611, -89.5175 43.0607)",link,LineString,24,0,"[-0.017108844593167305, -0.002981926314532757,...","[0.03018760494887829, 0.01594090275466442, -0....","[-0.01635056361556053, -0.001091119833290577, ...","[0.024474581703543663, 0.025692911818623543, -...","(LineString, contains, LineString)"


## Embedding-based method

1. get the embeddings of the spatial entities

2. train a classifier on spatial relations

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
relate_df['output'] = '(' + relate_df['geom_type_subject'] + ',' + relate_df['predicate'] + ',' + relate_df['geom_type'] + ')'

### text-embedding-ada-002

In [ ]:
model_abbr = 'ada'

In [ ]:
X_train = np.hstack([np.array(relations_train['embedding_subject_%s'%model_abbr].values.tolist()),
               np.array(relations_train['embedding_%s'%model_abbr].values.tolist())])
y_train = relations_train['rid'].values

X_test = np.hstack([np.array(relations_test['embedding_subject_%s'%model_abbr].values.tolist()),
               np.array(relations_test['embedding_%s'%model_abbr].values.tolist())])
y_test = relations_test['rid'].values

# train random forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
# probas = clf.predict_proba(X_test)




In [ ]:
res = relations_test[['subjectid', 'objectid', 'rid']]
res.loc[:, 'rid_%s'%model_abbr] = preds
res = pd.merge(res, relate_df[['rid', 'output']], on='rid', how='left')
res = pd.merge(res, relate_df[['rid', 'output']].rename(columns={'rid': 'rid_%s'%model_abbr,
                                                                 'output': 'output_%s'%model_abbr}),
               on='rid_%s'%model_abbr, how='left')
res.to_csv(path+'/result/task1/task1_%s.csv'%model_abbr, index=False)

In [ ]:
res = pd.read_csv(path+'/result/task1/task1_%s.csv'%model_abbr)
report = classification_report(res['rid'], res['rid_%s'%model_abbr], output_dict=True)
report = pd.DataFrame(report).T
score = report.iloc[-3:]
report = report.iloc[:-3].reset_index().rename(columns={'index': 'rid'})
score

,precision,recall,f1-score,support
accuracy,0.632857,0.632857,0.632857,0.632857
macro avg,0.631634,0.632857,0.622094,1400.000000
weighted avg,0.631634,0.632857,0.622094,1400.000000


In [ ]:
report['rid'] = report['rid'].astype(np.int32)
report = pd.merge(report, relate_df, on='rid', how='left')
report.sort_values(by='f1-score', ascending=False)

,rid,precision,recall,f1-score,support,geom_type_subject,predicate,geom_type,output
10,10,0.765957,0.900,0.827586,40.0,Point,touches,LineString,"(Point,touches,LineString)"
13,13,0.750000,0.825,0.785714,40.0,LineString,touches,Point,"(LineString,touches,Point)"
28,28,0.750000,0.825,0.785714,40.0,LineString,within,LineString,"(LineString,within,LineString)"
25,25,0.852941,0.725,0.783784,40.0,Polygon,touches,Point,"(Polygon,touches,Point)"
8,8,0.810811,0.750,0.779221,40.0,Point,touches,Polygon,"(Point,touches,Polygon)"
30,30,0.648148,0.875,0.744681,40.0,Point,within,LineString,"(Point,within,LineString)"
7,7,0.714286,0.750,0.731707,40.0,Point,within,Polygon,"(Point,within,Polygon)"
12,12,0.688889,0.775,0.729412,40.0,LineString,contains,Point,"(LineString,contains,Point)"
34,34,0.647059,0.825,0.725275,40.0,Polygon,contains,Polygon,"(Polygon,contains,Polygon)"
32,32,0.673913,0.775,0.720930,40.0,Polygon,equals,Polygon,"(Polygon,equals,Polygon)"


### text-embedding-3-large

In [ ]:
model_abbr = 'large'

In [ ]:
X_train = np.hstack([np.array(relations_train['embedding_subject_%s'%model_abbr].values.tolist()),
               np.array(relations_train['embedding_%s'%model_abbr].values.tolist())])
y_train = relations_train['rid'].values

X_test = np.hstack([np.array(relations_test['embedding_subject_%s'%model_abbr].values.tolist()),
               np.array(relations_test['embedding_%s'%model_abbr].values.tolist())])
y_test = relations_test['rid'].values

# train random forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
# probas = clf.predict_proba(X_test)




In [ ]:
res = relations_test[['subjectid', 'objectid', 'rid']]
res.loc[:, 'rid_%s'%model_abbr] = preds
res = pd.merge(res, relate_df[['rid', 'output']], on='rid', how='left')
res = pd.merge(res, relate_df[['rid', 'output']].rename(columns={'rid': 'rid_%s'%model_abbr,
                                                                 'output': 'output_%s'%model_abbr}),
               on='rid_%s'%model_abbr, how='left')
res.to_csv(path+'/result/task1/task1_%s.csv'%model_abbr, index=False)

In [ ]:
res = pd.read_csv(path+'/result/task1/task1_%s.csv'%model_abbr)
report = classification_report(res['rid'], res['rid_%s'%model_abbr], output_dict=True)
report = pd.DataFrame(report).T
score = report.iloc[-3:]
report = report.iloc[:-3].reset_index().rename(columns={'index': 'rid'})
score

,precision,recall,f1-score,support
accuracy,0.632143,0.632143,0.632143,0.632143
macro avg,0.626809,0.632143,0.622473,1400.000000
weighted avg,0.626809,0.632143,0.622473,1400.000000


In [ ]:
report['rid'] = report['rid'].astype(np.int32)
report = pd.merge(report, relate_df, on='rid', how='left')
report.sort_values(by='f1-score', ascending=False)

,rid,precision,recall,f1-score,support,geom_type_subject,predicate,geom_type,output
28,28,0.813953,0.875,0.843373,40.0,LineString,within,LineString,"(LineString,within,LineString)"
13,13,0.782609,0.900,0.837209,40.0,LineString,touches,Point,"(LineString,touches,Point)"
10,10,0.760870,0.875,0.813953,40.0,Point,touches,LineString,"(Point,touches,LineString)"
25,25,0.800000,0.700,0.746667,40.0,Polygon,touches,Point,"(Polygon,touches,Point)"
34,34,0.695652,0.800,0.744186,40.0,Polygon,contains,Polygon,"(Polygon,contains,Polygon)"
8,8,0.777778,0.700,0.736842,40.0,Point,touches,Polygon,"(Point,touches,Polygon)"
15,15,0.777778,0.700,0.736842,40.0,LineString,touches,Polygon,"(LineString,touches,Polygon)"
0,0,0.660000,0.825,0.733333,40.0,LineString,contains,LineString,"(LineString,contains,LineString)"
30,30,0.641509,0.850,0.731183,40.0,Point,within,LineString,"(Point,within,LineString)"
19,19,0.666667,0.800,0.727273,40.0,Polygon,within,Polygon,"(Polygon,within,Polygon)"
